In [2]:
# ==============================================================================
# 1_data_engineering_weather.ipynb
# ==============================================================================
# Título: Pipeline de Enriquecimento de Dados Meteorológicos (ETL)
# Autor: Time Data Science
# Descrição:
#   Este notebook executa o processo de ETL (Extract, Transform, Load) para
#   enriquecer o dataset de voos com dados climáticos históricos da API Open-Meteo.
#   A estratégia utiliza agrupamento por aeroporto/hora para minimizar chamadas à API
#   e garante a integridade dos dados através de tratamento de nulos e fuso horário.
# ==============================================================================

import pandas as pd
import numpy as np
import requests
import time
import os

# Configurações Globais
DATASET_VOOS = 'BrFlights2.csv'
DATASET_CLIMA_RAW = 'weather_history_2015_2017.csv'
DATASET_FINAL = 'BrFlights_Enriched_v4.csv'
API_URL = "https://archive-api.open-meteo.com/v1/archive"

# ==============================================================================
# 1. CARGA E PREPARAÇÃO INICIAL DOS DADOS DE VOOS
# ==============================================================================
print(f" Iniciando Pipeline de Engenharia de Dados...")

if not os.path.exists(DATASET_VOOS):
    print(f" Erro Crítico: Arquivo '{DATASET_VOOS}' não encontrado. Faça o upload.")
else:
    print(f" Carregando dataset de voos: {DATASET_VOOS}...")
    df = pd.read_csv(DATASET_VOOS, encoding='latin1', low_memory=False)

    # Seleção de colunas críticas para o georreferenciamento
    cols_geo = ['Aeroporto.Origem', 'LatOrig', 'LongOrig', 'Partida.Real']
    df_geo = df[cols_geo].dropna().copy()

    # Conversão para datetime e arredondamento para hora cheia (Join Key)
    df_geo['Partida.Real'] = pd.to_datetime(df_geo['Partida.Real'])
    df_geo['data_hora_clima'] = df_geo['Partida.Real'].dt.floor('h')

    print(f"✅ Dataset carregado. Total de registros: {len(df):,}")

# ==============================================================================
# 2. ANÁLISE DE PARETO E OTIMIZAÇÃO DE CHAMADAS
# ==============================================================================
print("\n Otimizando estratégia de extração (Pareto 90%)...")

# Identificar aeroportos que concentram 90% do tráfego
top_aeroportos = df['Aeroporto.Origem'].value_counts().reset_index()
top_aeroportos.columns = ['Aeroporto', 'Voos']
top_aeroportos['Acumulado'] = (top_aeroportos['Voos'] / top_aeroportos['Voos'].sum()).cumsum()

# Corte de 90%
lista_vips = top_aeroportos[top_aeroportos['Acumulado'] <= 0.90]['Aeroporto'].tolist()

print(f"   -> Total de Aeroportos: {len(top_aeroportos)}")
print(f"   -> Aeroportos selecionados (VIPs): {len(lista_vips)} (Cobrem 90% do tráfego)")

# Obter coordenadas únicas apenas para os VIPs
coords_vip = df[df['Aeroporto.Origem'].isin(lista_vips)] \
    .groupby('Aeroporto.Origem')[['LatOrig', 'LongOrig']] \
    .first() \
    .reset_index()

# ==============================================================================
# 3. EXTRAÇÃO DE DADOS (EXTRACT) - OPEN-METEO API
# ==============================================================================
# Verifica se já temos o arquivo de clima para não baixar tudo de novo
if os.path.exists(DATASET_CLIMA_RAW):
    print(f"\n Arquivo de clima '{DATASET_CLIMA_RAW}' já existe. Pulando download massivo.")
    df_weather_final = pd.read_csv(DATASET_CLIMA_RAW)
else:
    print(f"\n Iniciando download histórico (2015-2017) para {len(coords_vip)} aeroportos...")
    weather_dfs = []
    start_time = time.time()

    for index, row in coords_vip.iterrows():
        airport = row['Aeroporto.Origem']

        params = {
            "latitude": row['LatOrig'],
            "longitude": row['LongOrig'],
            "start_date": "2015-01-01",
            "end_date": "2017-12-31",
            "hourly": "precipitation,wind_speed_10m,visibility",
            "timezone": "UTC" # Crítico para alinhar com o dataset de voos
        }

        try:
            r = requests.get(API_URL, params=params)
            if r.status_code == 200:
                data = r.json()
                temp_df = pd.DataFrame({
                    'time': data['hourly']['time'],
                    'precipitation': data['hourly']['precipitation'],
                    'wind_speed': data['hourly']['wind_speed_10m'],
                    'visibility': data['hourly']['visibility']
                })
                temp_df['Aeroporto.Origem'] = airport
                weather_dfs.append(temp_df)
                print(f"   ✅ [{index+1}/{len(coords_vip)}] Baixado: {airport}")
            elif r.status_code == 429:
                print(f"    Rate Limit (429) em {airport}. Aguardando 10s...")
                time.sleep(10) # Backoff simples
            else:
                print(f"    Erro {r.status_code} em {airport}")

        except Exception as e:
            print(f"    Exceção em {airport}: {str(e)}")

        # Pausa de cortesia para a API
        time.sleep(0.5)

    if weather_dfs:
        df_weather_final = pd.concat(weather_dfs, ignore_index=True)
        df_weather_final.to_csv(DATASET_CLIMA_RAW, index=False)
        print(f"✅ Download concluído. Tempo total: {time.time() - start_time:.1f}s")
    else:
        print(" Falha crítica: Nenhum dado baixado.")
        # Em caso de falha real, interromper o notebook aqui seria ideal

# ==============================================================================
# 4. FUSÃO E TRATAMENTO (TRANSFORM)
# ==============================================================================
print(f"\n Executando Merge (Voo + Clima)...")

# Garantir tipos de dados compatíveis para o Join
df['Partida.Real'] = pd.to_datetime(df['Partida.Real'])
df['data_hora_merge'] = df['Partida.Real'].dt.floor('h')

df_weather_final['time'] = pd.to_datetime(df_weather_final['time'], utc=True)

# Join à esquerda (Left Join) para preservar todos os voos
df_enriched = df.merge(
    df_weather_final,
    left_on=['Aeroporto.Origem', 'data_hora_merge'],
    right_on=['Aeroporto.Origem', 'time'],
    how='left'
)

# Tratamento de Nulos (Imputação de Negócio)
# Premissa: Se não há dados de clima (aeroporto pequeno), assumimos "Boas Condições"
print(" Imputando valores nulos para aeroportos sem cobertura...")
fill_values = {
    'precipitation': 0.0,  # Sem chuva
    'wind_speed': df_enriched['wind_speed'].median(), # Vento médio
    'visibility': df_enriched['visibility'].max()     # Visibilidade máxima
}
df_enriched.fillna(value=fill_values, inplace=True)

# ==============================================================================
# 5. CARGA FINAL (LOAD)
# ==============================================================================
print(f"\n Salvando Dataset Mestre: {DATASET_FINAL}...")

# Selecionar colunas relevantes para o modelo (opcional, mas recomendado para reduzir tamanho)
# Aqui salvamos tudo para garantir flexibilidade
df_enriched.to_csv(DATASET_FINAL, index=False)

print("-" * 50)
print(f" PIPELINE DE ENGENHARIA CONCLUÍDO COM SUCESSO")
print(f"   -> Dataset Original: {df.shape}")
print(f"   -> Dataset Enriquecido: {df_enriched.shape}")
print(f"   -> Novas Features: precipitation, wind_speed, visibility")
print("-" * 50)

# Amostra final para validação visual
display(df_enriched[['Aeroporto.Origem', 'Partida.Real', 'precipitation', 'wind_speed']].head())

 Iniciando Pipeline de Engenharia de Dados...
 Carregando dataset de voos: BrFlights2.csv...
✅ Dataset carregado. Total de registros: 2,542,519

 Otimizando estratégia de extração (Pareto 90%)...
   -> Total de Aeroportos: 189
   -> Aeroportos selecionados (VIPs): 44 (Cobrem 90% do tráfego)

 Iniciando download histórico (2015-2017) para 44 aeroportos...
   ✅ [1/44] Baixado: Aeroporto Internacional Do Rio De Janeiro/Galeao
   ✅ [2/44] Baixado: Afonso Pena
   ✅ [3/44] Baixado: Bahia - Jorge Amado
   ✅ [4/44] Baixado: Brigadeiro Lysias Rodrigues
   ✅ [5/44] Baixado: Buenos Aires
   ✅ [6/44] Baixado: Buenos Aires/Aeroparque
   ✅ [7/44] Baixado: Campo Grande
   ✅ [8/44] Baixado: Cataratas
   ✅ [9/44] Baixado: Congonhas
   ✅ [10/44] Baixado: Deputado Luis Eduardo Magalhaes
   ✅ [11/44] Baixado: Eduardo Gomes
   ✅ [12/44] Baixado: Eurico De Aguiar Salles
   ✅ [13/44] Baixado: Governador Aluizio Alves
   ✅ [14/44] Baixado: Governador Jorge Teixeira De Oliveira
   ✅ [15/44] Baixado: Governador

/tmp/ipython-input-1995435917.py:152: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_enriched.fillna(value=fill_values, inplace=True)



 Salvando Dataset Mestre: BrFlights_Enriched_v4.csv...
--------------------------------------------------
 PIPELINE DE ENGENHARIA CONCLUÍDO COM SUCESSO
   -> Dataset Original: (2542519, 22)
   -> Dataset Enriquecido: (2542519, 26)
   -> Novas Features: precipitation, wind_speed, visibility
--------------------------------------------------


,Aeroporto.Origem,Partida.Real,precipitation,wind_speed
0,Afonso Pena,2016-01-30 08:58:00+00:00,0.0,9.1
1,Salgado Filho,2016-01-13 12:13:00+00:00,0.0,1.0
2,Salgado Filho,2016-01-29 12:13:00+00:00,0.0,6.0
3,Salgado Filho,2016-01-18 12:03:00+00:00,0.0,6.5
4,Salgado Filho,2016-01-30 12:13:00+00:00,0.0,5.9
